In [2]:
#boilerplate code for using classes to interact with SQL databases
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base #convert python classes into SQL tables
from sqlalchemy.ext.automap import automap_base #convert python classes into SQL tables
from sqlalchemy import Column, Integer, String, Float, text
import pymysql
from sqlalchemy.orm import Session
pymysql.install_as_MySQLdb()
from sqlalchemy import func
from sqlalchemy import distinct
import matplotlib.pyplot as plt
import numpy as np
from flask import Flask, jsonify

In [3]:
engine=create_engine("sqlite:///../hawaii.sqlite")

session=Session(bind=engine)

#Declare a Base useing automap_base()

Base = automap_base()

Base.prepare(engine, reflect=True)

Stations = Base.classes.stations
Measurements = Base.classes.measurements

In [31]:
import json
stations_query = session.query(Stations.station)
stations = pd.read_sql(stations_query.statement, stations_query.session.bind)
stations.to_dict()

{'station': {0: 'USC00519397',
  1: 'USC00513117',
  2: 'USC00514830',
  3: 'USC00517948',
  4: 'USC00518838',
  5: 'USC00519523',
  6: 'USC00519281',
  7: 'USC00511918',
  8: 'USC00516128'}}

In [2]:
#################################################
# Flask Setup
#################################################

from flask import Flask, jsonify

app = Flask(__name__)

#################################################
# Flask Routes
#################################################


'''
/api/v1.0/precipitation

Query for the dates and temperature observations from the last year.

Convert the query results to a Dictionary using date as the key and tobs as the value.

Return the json representation of your dictionary.

/api/v1.0/stations

Return a json list of stations from the dataset.
/api/v1.0/tobs

Return a json list of Temperature Observations (tobs) for the previous year

/api/v1.0/<start> and /api/v1.0/<start>/<end>

Return a json list of the minimum temperature, the average temperature, and the max temperature for a given start or start-end range.

When given the start only, calculate TMIN, TAVG, and TMAX for all dates greater than and equal to the start date.

When given the start and the end date, calculate the TMIN, TAVG, and TMAX for dates between the start and end date inclusive.
'''

@app.route("/api/v1.0/stations")
def stations():
    stations_query = session.query(Stations.station)
    stations = pd.read_sql(stations_query.statement, stations_query.session.bind)
    return(jsonify(stations))

@app.route("/api/v1.0/tobs")
def tobs():
    p12m_temp_query = session.query(Measurements).filter(Measurements.date>='2016-08-01',Measurements.date<'2017-08-01').with_entities(Measurements.date, Measurements.station, Measurements.tobs)
    p12m_temp = pd.read_sql(p12m_temp_query.statement, p12m_temp_query.session.bind)
    return(jsonify(p12m_temp))

@app.route("/api/v1.0/justice-league/<start>")
def temps(start):
    temp_query = session.query(Measurements).filter(Measurements.date>=start).with_entities(Measurements.date, Measurements.tobs)
    df = pd.read_sql(temp_query.statement, temp_query.session.bind)
    TMIN=df['tobs'].min()
    TMAX=df['tobs'].max()
    TAVG=df['tobs'].mean()
    results=[TMIN,TMAX,TAVG]
    return jsonify(results)

@app.route("/api/v1.0/justice-league/<start>/<end>")
def temps(start,end):
    temp_query = session.query(Measurements).filter(Measurements.date>=start,Measurements.date<=end).with_entities(Measurements.date, Measurements.tobs)
    df = pd.read_sql(temp_query.statement, temp_query.session.bind)
    TMIN=df['tobs'].min()
    TMAX=df['tobs'].max()
    TAVG=df['tobs'].mean()
    results=[TMIN,TMAX,TAVG]
    return jsonify(results)

#@TODO: Test all this

if __name__ == "__main__":
    app.run(debug=True)
    raise NotImplementedError()




#see what's poppin by going to localhost:5000
#when you run the app, it will tell you:
#Running on http://127.0.0.1:5000/

#that's how you know the port is 5000 (replace 127.0.0.1 with localhost)